In [27]:
#importing libraries
import os
import re, string, unicodedata
import nltk
import contractions
import inflect
from bs4 import BeautifulSoup
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
#from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import glob
import numpy as np
import datetime
import pickle
from gensim.models.fasttext import FastText

#Used to remove html markups, etc

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)


def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text


#Functions used for preprocessing textual data

def replace_contractions(text):
    """Replaces contractions (it's -> it is)"""
    return contractions.fix(text)


def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    words = list(map(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf-8', 'ignore'), words))
    return words



def to_lowercase(words):
    words = list(map(lambda x: x.lower(), words))
    return words



def remove_punctuation(words):
    words = list(map(lambda x: re.sub(r'[^\w\s]', '', x), words))
    words = list(filter(None, words))#removing empty strings after punctuation removal
    return words



def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    words = [word for word in words if not word in stopwords.words('english')]
    return words



def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    words = list(map(lambda x: lemmatizer.lemmatize(x,pos='v'), words))
    return words



def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = lemmatize_verbs(words)
    return words



#Function that tracks occurrences of multi-worded terms


def mult_replace(text):
    dict = {
        "pro-rights": "prorights",
        "heart attack": "heartattack",
        "yellow-skinned": "yellowskinned",
        "multi-racial": "multiracial",
        "african-american": "africanamerican",
        "afro-american": "afroamerican",
        "non-white": "nonwhite",
        "middle-east": "middleeast",
        "middle-eastern": "middleeastern",
        "cancer patient": "cancerpatient",
        "drug addict": "drugaddict",
        "split-personality": "splitpersonality",
        "Pro-rights": "prorights",
        "Heart attack": "heartattack",
        "Yellow-skinned": "yellowskinned",
        "Multi-racial": "multiracial",
        "African American": "africanamerican",
        "Afro-American": "afroamerican",
        "Non-white": "nonwhite",
        "Middle-east": "middleeast",
        "Middle-eastern": "middleeastern",
        "Cancer patient": "cancerpatient",
        "Drug addict": "drugaddict",
        "Split-personality": "splitpersonality",
        "ex-criminal": "excriminal",
        "Ex-criminal": "excriminal",
        "blue-collar": "bluecollar",
        "Blue-collar": "bluecollar",
        "working-class": "workingclass",
        "Working-class": "workingclass",
        "Native American": "nativeamerican",
        "Indo-Aryan": "indoaryan",
        "Cross-dressing": "crossdressing",
        "cross-dressing": "crossdressing"
    }
    
    regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))
    
    return regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text)

    


In [28]:
file_list = []

file_list = glob.glob("/media/nishanthsanjeev/2CC66AE5C66AAF30/Embeddings/COCA/Text Files/*/*1997.txt")

In [29]:
file_list

['/media/nishanthsanjeev/2CC66AE5C66AAF30/Embeddings/COCA/Text Files/text_acad_isi/text_acad_1997.txt',
 '/media/nishanthsanjeev/2CC66AE5C66AAF30/Embeddings/COCA/Text Files/text_fic_jjw/text_fic_1997.txt',
 '/media/nishanthsanjeev/2CC66AE5C66AAF30/Embeddings/COCA/Text Files/text_mag_jkf/text_mag_1997.txt',
 '/media/nishanthsanjeev/2CC66AE5C66AAF30/Embeddings/COCA/Text Files/text_news_nne/text_news_1997.txt',
 '/media/nishanthsanjeev/2CC66AE5C66AAF30/Embeddings/COCA/Text Files/text_spok_yuv/text_spok_1997.txt',
 '/media/nishanthsanjeev/2CC66AE5C66AAF30/Embeddings/COCA/Text Files/text_tvm_nwh/text_tvm_1997.txt']

In [30]:
#Generating the new, combined text file.


with open('coca_1997.txt', 'w') as outfile:
    for names in file_list:
        print(names)
        with open(names,'r') as infile:
            outfile.write(infile.read())
            
        outfile.write("\n")

/media/nishanthsanjeev/2CC66AE5C66AAF30/Embeddings/COCA/Text Files/text_acad_isi/text_acad_1997.txt
/media/nishanthsanjeev/2CC66AE5C66AAF30/Embeddings/COCA/Text Files/text_fic_jjw/text_fic_1997.txt
/media/nishanthsanjeev/2CC66AE5C66AAF30/Embeddings/COCA/Text Files/text_mag_jkf/text_mag_1997.txt
/media/nishanthsanjeev/2CC66AE5C66AAF30/Embeddings/COCA/Text Files/text_news_nne/text_news_1997.txt
/media/nishanthsanjeev/2CC66AE5C66AAF30/Embeddings/COCA/Text Files/text_spok_yuv/text_spok_1997.txt
/media/nishanthsanjeev/2CC66AE5C66AAF30/Embeddings/COCA/Text Files/text_tvm_nwh/text_tvm_1997.txt


In [31]:
#begin_time = datetime.datetime.now()

with open('coca_1997.txt','r') as f:
    strt = f.read()
    
str1 = mult_replace(strt)
del strt

comb_cl = re.sub("\\[']\d*","'",str1)
del str1

comb_cl_denoised = denoise_text(comb_cl)
del comb_cl
print(datetime.datetime.now() - begin_time)
comb_cl_denoised = comb_cl_denoised.replace('@','')

comb_cl_denoised_new = comb_cl_denoised.replace(" '","'")
del comb_cl_denoised

comb_cl_den_new2 = comb_cl_denoised_new.replace(" n't","n't")
del comb_cl_denoised_new


comb_cl_new3 = comb_cl_den_new2.replace("-"," ")
del comb_cl_den_new2


comb_cl_d = comb_cl_new3.replace("\"","")

sentences = nltk.sent_tokenize(comb_cl_d)
del comb_cl_d

sentences[:5]

["\n4000054 Section : Features Research sheds new light on the origin of humanity's most intimate quadruped ally  The poor dog ,  wrote poet Lord Byron in a flight of emotion ,  in life the firmest friend , The first to welcome , foremost to defend .",
 'And certainly , few animal lovers would care to differ .',
 "The dog , after all , is commonly referred to as man's best friend , and unquestionably serves a wide range of human purposes .",
 'Thanks to artificial selection , there are dogs that guard houses and dogs that herd livestock , dogs that locate game birds for shooting and dogs that retrieve game birds that have been shot , dogs that pull sleds and dogs that sit languidly in human laps .',
 'Clearly , the relationship between dog and human runs deep in our culture and our psyches .']

In [32]:
#Replacing contractions, i.e. it's -> it is, et cetera


sentences_mod = list(map(lambda x: replace_contractions(x), sentences))    
del sentences


words = list(map(lambda x: nltk.word_tokenize(x), sentences_mod))    
    
del sentences_mod

In [34]:
#words[:5]

In [35]:
#train_data = normalize(words)
train_data = list(map(lambda x: normalize(x), words))    
del words
train_data[:5]

[['4000054',
  'section',
  'feature',
  'research',
  'shed',
  'new',
  'light',
  'on',
  'the',
  'origin',
  'of',
  'humanity',
  's',
  'most',
  'intimate',
  'quadruped',
  'ally',
  'the',
  'poor',
  'dog',
  'write',
  'poet',
  'lord',
  'byron',
  'in',
  'a',
  'flight',
  'of',
  'emotion',
  'in',
  'life',
  'the',
  'firmest',
  'friend',
  'the',
  'first',
  'to',
  'welcome',
  'foremost',
  'to',
  'defend'],
 ['and',
  'certainly',
  'few',
  'animal',
  'lovers',
  'would',
  'care',
  'to',
  'differ'],
 ['the',
  'dog',
  'after',
  'all',
  'be',
  'commonly',
  'refer',
  'to',
  'as',
  'man',
  's',
  'best',
  'friend',
  'and',
  'unquestionably',
  'serve',
  'a',
  'wide',
  'range',
  'of',
  'human',
  'purpose'],
 ['thank',
  'to',
  'artificial',
  'selection',
  'there',
  'be',
  'dog',
  'that',
  'guard',
  'house',
  'and',
  'dog',
  'that',
  'herd',
  'livestock',
  'dog',
  'that',
  'locate',
  'game',
  'bird',
  'for',
  'shoot',
  'an

In [36]:
with open('/home/nishanthsanjeev/Harvard/PickledCOCA/train1997','wb') as f:
    pickle.dump(train_data,f)

In [19]:
# import pickle

# with open('/home/nishanthsanjeev/Harvard/PickledCOCA/train1993','rb') as fr:
#     train_data = pickle.load(fr)
    

In [ ]:
embedding_size = 100
window_size = 40
min_word = 5
down_sampling = 1e-2

begin_time = datetime.datetime.now()

ft_model = FastText(train_data,
                      size=embedding_size,
                      window=window_size,
                      min_count=min_word,
                      sample=down_sampling,
                    sg=1,
                      iter=10,
                   workers = 6)


print(datetime.datetime.now() - begin_time)#time taken to train this set of vectors

In [23]:
semantically_similar_words = {words: [item[0] for item in ft_model.wv.most_similar([words], topn=5)]
                  for words in ['kitchen', 'death', 'king', 'queen', 'strong', 'weak','woman','man']}

for k,v in semantically_similar_words.items():
    print(k+":"+str(v))

kitchen:['kitchenette', 'bathroom', 'upstairs', 'downstairs', 'breakfast']
death:['torture', 'dead', 'murder', 'innocent', 'mortally']
king:['larry', 'peter', 'jester', 'sain', 'haing']
queen:['queenie', 'princesse', 'princess', 'royal', 'bessette']
strong:['clearly', 'tough', 'hard', 'trouble', 'surely']
weak:['negatively', 'weakness', 'weaker', 'weakling', 'negative']
woman:['girl', 'daughter', 'child', 'she', 'matronly']
man:['one', 'ever', 'way', 'who', 'thing']


In [24]:
ft_model.wv.save_word2vec_format("/home/nishanthsanjeev/Harvard/COCA_Vectors_Year/coca_vector1997.txt")

In [23]:
#ft_model = KeyedVectors.load_word2vec_format("coca_vector1991.txt")